###A Decorator is a callable that takes a callable as input and returns another callable
Decorators allow you to decorate or wrap a function and lets you execute code before or after the wrapped function runs

You can define reusable blocks without permanently modifying the the wrapped function itself




In [38]:
from datetime import datetime
def null_decorator(func):
  """
  A callable that takes another callable as input and returns it without modiying it.
  """
  def new_func(text):
    print(datetime.now().time())
    val = func(text)
    print(val)
    print(datetime.now().time())
  return new_func

def greet(text):
  return text + '!'

# The greet function is replaced by new_func() returned by null_decorator. This is exactly what applying the decorator does
greet = null_decorator(greet)
greet('Hello')
print('')


@null_decorator
def greet_1(text):
  return text + '_!'

greet_1('Hello')



11:33:13.752883
Hello!
11:33:13.753595

11:33:13.753858
Hello_!
11:33:13.754069


greet = null_decorator(greet) is the same as applying @null_decorator on the greet function

In [58]:
print('Multiple decorators')

def add_x(func):
  def wrapper():
    return 'x' + func() + 'x'
  return wrapper

def add_y(func):
  def wrapper():
    return 'y' + func() + 'y'
  return wrapper

@add_x
@add_y
def greet():
  return 'abc'

print(greet())

# This is the same as calling greet = add_x(add_y(greet))
greet1 = add_x(add_y(greet))()
print(greet1)


Multiple decorators
xyabcyx
xyxyabcyxyx


Multiple decorators are executed **bottom to top**.  The topmost decorator will be applied last.